In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats

In [ ]:
bond  = pd.read_csv("../data/raw/Isin.csv")

In [ ]:
bond.loc[~bond.Seniority.isin(["GOV","SEC","SEN","SUB"]), "Seniority"] = "OTHER"
bond.loc[~bond.Activity.isin(["Asia","RETAIL","GBP SAS", "ARGENTINIA"]), "Activity"] = "OTHER"
bond.loc[bond.Currency.isin(
    (bond.groupby("Currency").size())[bond.groupby("Currency").size()<350].index.tolist()
), "Currency"] = "OTHER"

bond.loc[bond.CompositeRating.str.contains("C|D"), "CompositeRating"] = "LOWER"
# Risk captain
bond.loc[bond.RiskCaptain.isin(
    (bond.groupby("RiskCaptain").size())[bond.groupby("RiskCaptain").size()<200].index.tolist()
), "RiskCaptain"] = "OTHER"

In [ ]:
data = pd.read_csv("../data/prepared/data_basic.csv", compression = "gzip")

In [ ]:
tmp = data.loc[data.CustomerInterest == 1]

tmp = tmp.loc[tmp.Week < 61]

full = tmp.merge(bond, on = ["IsinIdx"], how = "left")

In [ ]:
for var in full.columns:
    full[var].fillna("Other", inplace = True)

In [ ]:
fac_data = full.groupby("CustomerIdx").agg(lambda x: scipy.stats.mode(x)[0][0])
fac_data.columns = fac_data.columns + "_mode"
fac_data = fac_data.reset_index()
fac_data.head()

In [ ]:
# list of features
excluded_features = ["PredictionIdx_mode", "CustomerInterest_mode", "Week_mode", 
                     "ActualMaturityDateKey_mode", "IssueDateKey_mode", "IssuedAmount_mode"]
features = [var for var in fac_data.columns if var not in excluded_features]
fac_data = fac_data[features]
fac_data.head()

In [ ]:
fac_data.to_csv("../data/prepared/favorite_bonds_v3_60.csv", index = False, compression = "gzip")